# Process Election Results

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/dcboe/November_3_2020_General_Election_Pre-certified_Results.csv')
df = df.rename(columns={
    'Candidate': 'candidate_name'
    , 'ContestName': 'contest_name'
    , 'ContestNumber': 'contest_number'
    , 'Votes': 'votes'
})

In [3]:
df[df['candidate_name'] == 'DEM Joseph R. Biden - Kamala D. Harris'].votes.sum()

317323

In [4]:
anc = df[df['contest_name'].str.contains('SINGLE MEMBER DISTRICT')].copy()

In [5]:
anc.groupby('contest_name').votes.sum()

contest_name
ANC - 1A01 SINGLE MEMBER DISTRICT  01-ANC 1A     519
ANC - 1A02 SINGLE MEMBER DISTRICT  02-ANC 1A     797
ANC - 1A03 SINGLE MEMBER DISTRICT  03-ANC 1A     616
ANC - 1A04 SINGLE MEMBER DISTRICT  04-ANC 1A     705
ANC - 1A05 SINGLE MEMBER DISTRICT  05-ANC 1A     631
                                                ... 
ANC - 8E03 SINGLE MEMBER DISTRICT  03-ANC 8E     104
ANC - 8E04 SINGLE MEMBER DISTRICT  04-ANC 8E     866
ANC - 8E05 SINGLE MEMBER DISTRICT  05-ANC 8E    1049
ANC - 8E06 SINGLE MEMBER DISTRICT  06-ANC 8E     849
ANC - 8E07 SINGLE MEMBER DISTRICT  07-ANC 8E     907
Name: votes, Length: 296, dtype: int64

In [6]:
anc['smd_id'] = 'smd_' + anc['contest_name'].str[6:10]

In [7]:
anc[anc['smd_id'] == 'smd_1C07']

,ElectionDate,ElectionName,contest_number,contest_name,PrecinctNumber,WardNumber,candidate_name,Party,votes,smd_id
1776,11/3/2020 12:00:00 AM,General Election,48,ANC - 1C07 SINGLE MEMBER DISTRICT 07-ANC 1C,24,1,Japer Bowles,NPN,728,smd_1C07
1777,11/3/2020 12:00:00 AM,General Election,48,ANC - 1C07 SINGLE MEMBER DISTRICT 07-ANC 1C,24,1,Write-in,NPN,46,smd_1C07
2550,11/3/2020 12:00:00 AM,General Election,48,ANC - 1C07 SINGLE MEMBER DISTRICT 07-ANC 1C,35,1,Japer Bowles,NPN,192,smd_1C07
2551,11/3/2020 12:00:00 AM,General Election,48,ANC - 1C07 SINGLE MEMBER DISTRICT 07-ANC 1C,35,1,Write-in,NPN,22,smd_1C07


In [8]:
candidates = anc.groupby(['smd_id', 'contest_number', 'candidate_name']).votes.sum().reset_index()
candidates

,smd_id,contest_number,candidate_name,votes
0,smd_1A01,18,Layla Bonnot,501
1,smd_1A01,18,Write-in,18
2,smd_1A02,19,Dieter Lehmann Morales,761
3,smd_1A02,19,Write-in,36
4,smd_1A03,20,Write-in,18
...,...,...,...,...
693,smd_8E06,312,Karlene (K.) Armstead,317
694,smd_8E06,312,Write-in,22
695,smd_8E07,313,"Jeannina ""Ward 8 Matters"" Williams",721
696,smd_8E07,313,Stephen A. Slaughter,177


In [9]:
import hashlib
def hash_dataframe(df, columns_to_hash):
    """
    Given a DataFrame, hash certain columns

    df = pandas DataFrame
    columns_to_hash = a list containing the column names that should be hashed
    """

    hash_of_data = []

    for idx, row in df.iterrows():
        list_to_hash = row[columns_to_hash]
        string_to_hash = ','.join(list_to_hash)
        hash_of_data += [hashlib.sha224(string_to_hash.encode()).hexdigest()]
    
    return hash_of_data


In [10]:
candidates['dcboe_hash_id'] = hash_dataframe(candidates, ['smd_id', 'candidate_name'])

In [11]:
candidates['ranking'] = candidates.groupby('smd_id').votes.rank(method='first', ascending=False)
candidates['winner'] = candidates['ranking'] == 1

In [12]:
candidates['write_in_winner'] = candidates['winner'] & (candidates['candidate_name'] == 'Write-in')

In [13]:
candidates = candidates.sort_values(by=['smd_id', 'votes'], ascending=[True, False])

In [14]:
total_votes = candidates.groupby('smd_id').votes.sum()
total_votes.name = 'total_votes'
candidates = pd.merge(candidates, total_votes, how='inner', on='smd_id')
candidates['vote_share'] = candidates['votes'] / candidates['total_votes']

In [15]:
shift_one = candidates[['smd_id', 'votes', 'vote_share']].shift(-1)
shift_one = shift_one.rename(columns={
    'smd_id': 'smd_id_shifted'
    , 'votes': 'votes_shifted'
    , 'vote_share': 'vote_share_shifted'
})

candidates = pd.concat([candidates, shift_one], axis=1)

In [16]:
candidates['margin_of_victory'] = None
candidates['margin_of_victory_percentage'] = None
contested_winners = (candidates['smd_id'] == candidates['smd_id_shifted']) & (candidates['winner'])

candidates.loc[contested_winners, 'margin_of_victory'] = candidates['votes'] - candidates['votes_shifted']
candidates.loc[contested_winners, 'margin_of_victory_percentage'] = candidates['vote_share'] - candidates['vote_share_shifted']


In [17]:
num_candidates = candidates.groupby('smd_id').candidate_name.count()
num_candidates.name = 'num_candidates'
candidates = pd.merge(candidates, num_candidates, how='inner', on='smd_id')

In [18]:
candidates[[
    'dcboe_hash_id'
    , 'smd_id'
    , 'contest_number'
    , 'candidate_name'
    , 'votes'
    , 'vote_share'
    , 'ranking'
    , 'winner'
    , 'write_in_winner'
    , 'margin_of_victory'
    , 'margin_of_victory_percentage'
    , 'num_candidates'
    , 'total_votes'
]].to_csv('../data/dcboe/candidate_votes.csv', index=False)

In [19]:
candidates.sort_values(by='margin_of_victory_percentage').head(20)

,smd_id,contest_number,candidate_name,votes,dcboe_hash_id,ranking,winner,write_in_winner,total_votes,vote_share,smd_id_shifted,votes_shifted,vote_share_shifted,margin_of_victory,margin_of_victory_percentage,num_candidates
522,smd_7B01,244,Chioma J. Iwuoha,515,5c3624c4c0a33fd119712d05ce5b6143d85141ad71498f...,1.0,True,False,1039,0.495669,smd_7B01,513.0,0.493744,2,0.00192493,3
431,smd_5E10,205,Sally Hobaugh,636,87c0a433075547ef887b20a4ff5682ad199b65aa0b8d45...,1.0,True,False,1270,0.500787,smd_5E10,627.0,0.493701,9,0.00708661,3
628,smd_8A07,285,Steven Tiller,367,53027debfd0d0d7788549943a62fa7db6f8b2e4fde75d5...,1.0,True,False,1093,0.335773,smd_8A07,359.0,0.328454,8,0.0073193,4
634,smd_8B03,288,Charles H. Wilson,299,9a72c179f4b4c0aef1f12f8e24d7e330be6298706889e4...,1.0,True,False,864,0.346065,smd_8B03,287.0,0.332176,12,0.0138889,4
130,smd_2E02,78,Jenny Mitchell,276,f51d42bb8b3eefa298538a5c9b84ec7fa048a881233df4...,1.0,True,False,546,0.505495,smd_2E02,262.0,0.479853,14,0.025641,3
366,smd_5B05,181,Colleen Costello,604,0ab458e64c2af7777f9e9e5b0417e1f6ff1831e93c98dc...,1.0,True,False,1205,0.501245,smd_5B05,568.0,0.471369,36,0.0298755,3
324,smd_4C10,162,Jonah Goodman,560,2dfc6d733bb4cf95c4ceb2da45fa3243525bc0441f1182...,1.0,True,False,1105,0.506787,smd_4C10,524.0,0.474208,36,0.0325792,3
182,smd_3C05,102,Sauleh A Siddiqui,577,94bcd86cb01050f6369358fdfe415ac7d6c0a891b14c8a...,1.0,True,False,1201,0.480433,smd_3C05,533.0,0.443797,44,0.0366361,4
580,smd_7E04,268,Natasha Dupee,486,deb5e54ddfd46cd4e2cb518700d5faf834a9c3197d34c8...,1.0,True,False,1146,0.424084,smd_7E04,444.0,0.387435,42,0.0366492,4
398,smd_5D05,193,Sydelle Moore,432,f7dd6fc22c57dd50b15e0db3c07218fe1af71454a93e90...,1.0,True,False,838,0.515513,smd_5D05,396.0,0.472554,36,0.0429594,3


In [20]:
# candidates[candidates['write_in_winner']] #  & (candidates['num_candidates'] > 1)]

In [21]:
candidates.sort_values(by='votes', ascending=False).head(10)

,smd_id,contest_number,candidate_name,votes,dcboe_hash_id,ranking,winner,write_in_winner,total_votes,vote_share,smd_id_shifted,votes_shifted,vote_share_shifted,margin_of_victory,margin_of_victory_percentage,num_candidates
489,smd_6C06,229,Drew Courtney,2848,014cac4dd905f3c85a0b269bf43a352ebe04c08b102b7a...,1.0,True,False,2945,0.967063,smd_6C06,97.0,0.032937,2751,0.934126,2
501,smd_6D07,236,Edward Daniels,2053,e03a1c254580e46525a9b4d6ef204bab0f12b224588086...,1.0,True,False,3116,0.658858,smd_6D07,985.0,0.316110,1068,0.342747,3
515,smd_6E05,241,Patrick K. Parlej,1573,826fafb76a69665a2585610f5f679c7a0fb1ec0de5548c...,1.0,True,False,1634,0.962668,smd_6E05,61.0,0.037332,1512,0.925337,2
375,smd_5C03,184,Pierre Hines,1467,d1b97aea73c2cbd2eecec59929bb5008ac235d0836e983...,1.0,True,False,1523,0.963230,smd_5C03,56.0,0.036770,1411,0.926461,2
487,smd_6C05,228,Joel Kelty,1354,84edb33a0a2bed21dd3e52fbee27bae82485e5deaeaefc...,1.0,True,False,1430,0.946853,smd_6C05,76.0,0.053147,1278,0.893706,2
495,smd_6D04,233,Andy Litsky,1316,e4d3a374dfcf85028220dbf0963ec9ff43e9e5121beecc...,1.0,True,False,1369,0.961286,smd_6D04,53.0,0.038714,1263,0.922571,2
291,smd_4B07,150,Geoff Bromaghim,1242,74ddb5ca35a50e07e4dff3aea623bbc42a674759ea3584...,1.0,True,False,1300,0.955385,smd_4B07,58.0,0.044615,1184,0.910769,2
468,smd_6B06,219,Corey Holman,1186,1517ed5f34ee8200194cefac2d575ead1ecae8d2b3dc14...,1.0,True,False,1227,0.966585,smd_6B06,41.0,0.033415,1145,0.93317,2
497,smd_6D05,234,"Fredrica (""Rikki"") Kramer",1178,154de7e1905ffd769138b98f0df6b1de66c0554772b9b2...,1.0,True,False,1232,0.956169,smd_6D05,54.0,0.043831,1124,0.912338,2
364,smd_5B04,180,Ra Amin,1162,a36c6cd0011c818e5ca0cee688bc255fb40c2c517906c2...,1.0,True,False,1212,0.958746,smd_5B04,50.0,0.041254,1112,0.917492,2


In [22]:
anc[anc['smd_id'] == 'smd_6C06']

,ElectionDate,ElectionName,contest_number,contest_name,PrecinctNumber,WardNumber,candidate_name,Party,votes,smd_id
5983,11/3/2020 12:00:00 AM,General Election,229,ANC - 6C06 SINGLE MEMBER DISTRICT 06-ANC 6C,83,6,Drew Courtney,NPN,769,smd_6C06
5984,11/3/2020 12:00:00 AM,General Election,229,ANC - 6C06 SINGLE MEMBER DISTRICT 06-ANC 6C,83,6,Write-in,NPN,33,smd_6C06
10498,11/3/2020 12:00:00 AM,General Election,229,ANC - 6C06 SINGLE MEMBER DISTRICT 06-ANC 6C,144,6,Write-in,NPN,64,smd_6C06
10499,11/3/2020 12:00:00 AM,General Election,229,ANC - 6C06 SINGLE MEMBER DISTRICT 06-ANC 6C,144,6,Drew Courtney,NPN,2079,smd_6C06
